In [81]:
import netCDF4
import numpy as np

In [82]:
from netCDF4 import Dataset
rootgrp0 = Dataset("KMDS__OPER_P___10M_OBS_L2_202505310920.nc", "r")

In [83]:
rootgrp0.ncattrs()

['featureType',
 'Conventions',
 'title',
 'institution',
 'source',
 'history',
 'references',
 'comment']

In [84]:
rootgrp0.variables

{'station': <class 'netCDF4.Variable'>
 vlen station(station)
     long_name: Station id
     cf_role: timeseries_id
 vlen data type: <class 'str'>
 unlimited dimensions: 
 current shape = (69,),
 'time': <class 'netCDF4.Variable'>
 float64 time(time)
     long_name: time of measurement
     standard_name: time
     units: seconds since 1950-01-01 00:00:00
 unlimited dimensions: 
 current shape = (1,)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 'wsi': <class 'netCDF4.Variable'>
 vlen wsi(station)
     long_name: Station wsi
 vlen data type: <class 'str'>
 unlimited dimensions: 
 current shape = (69,),
 'stationname': <class 'netCDF4.Variable'>
 vlen stationname(station)
     long_name: Station name
 vlen data type: <class 'str'>
 unlimited dimensions: 
 current shape = (69,),
 'lat': <class 'netCDF4.Variable'>
 float64 lat(station)
     long_name: station  latitude
     standard_name: latitude
     units: degrees_north
 unlimited dimensions: 
 current shape = (69,)


In [85]:
aaovnwpm=np.array(['dd','dn','dsd', 'dx', 'fsd', 'gff', 'hc1', 'pp', 'ps', 'Q1H', 'Q24H', 'qgn', 'qgx', 'rh', 'Sax6H', 'ss', 'Sx6H', 'tg', 'tgn', 'Tgn12'])#array of variable names with proper mask

Rоличество переменных, для которых нет данных на метеостанции(для каждой метеостанции):

In [86]:
np.sum(np.array([np.ma.getmask(rootgrp0[name][:]) for name in aovnwpm]), axis=0)

array([[ 9],
       [ 8],
       [ 8],
       [ 9],
       [ 8],
       [15],
       [ 8],
       [ 8],
       [ 0],
       [ 8],
       [ 8],
       [12],
       [12],
       [ 8],
       [ 0],
       [ 8],
       [ 8],
       [ 8],
       [ 8],
       [ 4],
       [ 5],
       [12],
       [ 3],
       [ 1],
       [20],
       [11],
       [12],
       [ 0],
       [ 2],
       [ 0],
       [ 0],
       [ 3],
       [ 0],
       [ 3],
       [ 3],
       [ 0],
       [ 0],
       [ 3],
       [16],
       [ 3],
       [ 0],
       [16],
       [ 0],
       [16],
       [16],
       [16],
       [16],
       [10],
       [ 1],
       [13],
       [13],
       [ 1],
       [16],
       [ 1],
       [16],
       [ 5],
       [12],
       [ 4],
       [ 1],
       [ 0],
       [ 1],
       [ 0],
       [ 0],
       [ 2],
       [ 4],
       [ 3],
       [ 1],
       [ 1],
       [ 0]])

Количество метеостанций, для которых нет данных по переменной, для каждой переменной:

In [87]:
np.sum(np.array([np.ma.getmask(rootgrp0[name][:]) for name in aovnwpm]), axis=1)

array([[ 2],
       [15],
       [10],
       [15],
       [13],
       [ 2],
       [36],
       [29],
       [29],
       [30],
       [34],
       [30],
       [30],
       [22],
       [ 5],
       [34],
       [ 5],
       [32],
       [33],
       [33]])

Я отбираю станции, у которых отсутствует не более 1 переменной из нашего набора переменных:

In [88]:
missingstdata = np.sum(np.array([np.ma.getmask(rootgrp0[name][:]) for name in aovnwpm]), axis=0)
fittingsts = []
for i in range(len(missingstdata)):
    if missingstdata[i] <= 1:
        fittingsts.append(i)
fittingsts=np.array(fittingsts)
fittingsts.shape

(22,)

Я отбираю переменные, которые есть во всех отобранных прошлым шагом станциях:

In [91]:
missingvardata = np.sum(np.array([[np.ma.getmask(rootgrp0[name][s][0]) for s in fittingsts] for name in aovnwpm]), axis=1)
print(missingvardata)
fittingvrs = []
for i in range(len(missingvardata)):
    if missingvardata[i] == 0:
        fittingvrs.append(i)
fittingvrs=np.array(fittingvrs)
fittingvrs.shape

[0 0 0 0 0 0 8 0 0 0 0 0 0 0 0 0 0 0 0 0]


(19,)

Я беру данные по отобранным станциям и отобранным переменным. Также я умножаю некоторые переменные на константу, чтобы все переменные были примерно одного порядка. Иначе BHEP-тест не работает:

In [92]:
print(aovnwpm[fittingvrs])
varcall=aovnwpm[fittingvrs]#список имен отобранных переменных
var_mult={}
for name in varcall:
    var_mult[name]=1
for name in['Q24H', 'qgn', 'qgx']:
    var_mult[name]/=100
for name in ['dd', 'dn', 'dx', 'dsd', 'Q1H']:
    var_mult[name]/=10
for name in ['fsd', 'dsd']:
    var_mult[name]*=10
print(var_mult)
data=np.swapaxes(np.array([np.array([np.ma.getdata(rootgrp0[name][s][0])for s in fittingsts])*var_mult[name] for name in varcall]), 0, 1)
data.shape

['dd' 'dn' 'dsd' 'dx' 'fsd' 'gff' 'pp' 'ps' 'Q1H' 'Q24H' 'qgn' 'qgx' 'rh'
 'Sax6H' 'ss' 'Sx6H' 'tg' 'tgn' 'Tgn12']
{'dd': 0.1, 'dn': 0.1, 'dsd': 1.0, 'dx': 0.1, 'fsd': 10, 'gff': 1, 'pp': 1, 'ps': 1, 'Q1H': 0.1, 'Q24H': 0.01, 'qgn': 0.01, 'qgx': 0.01, 'rh': 1, 'Sax6H': 1, 'ss': 1, 'Sx6H': 1, 'tg': 1, 'tgn': 1, 'Tgn12': 1}


(22, 19)

Пример данных по одной из переменных:

In [93]:
data[:,1]

array([17.58, 19.97, 17.86,  9.42,  4.92, 14.34, 12.66, 18.28, 15.75,
       10.69, 20.53,  6.75, 17.16, 18.28, 13.22, 17.3 , 10.13, 12.09,
       11.53,  8.58,  5.48,  7.03])

Документация по отобранным переменным:

In [94]:
for name in varcall:
    print(rootgrp0[name])

<class 'netCDF4.Variable'>
float64 dd(station, time)
    _FillValue: -9999.0
    standard_name: wind_from_direction
    units: degree
    long_name: Wind Direction 10 Min Average with MD
    comment: MD: Marked Discontinuity is an abrupt change in the 2-minute average wind speed and/or direction. This is defined per WMO and ICAO guidelines as more than 10 knots and/or 30 degrees absolute difference between two successive 2-minute periods  when the wind speed > 10 knots in one or both periods. When a MD is detected, all data older than 2 minutes are excluded from calculation. Variables indicated with MD are therefore calculated based on the samples measured within the previous 2 minutes at minimum, up to the full 10-minute period. Note: For wind observations of stations 06209, 06285, 06308, 06312, 06313, 06315, 06316, 06324, 06331 the timestamp indicates the middle of the interval. At other locations the timestamp of the wind observations indicates the end of the interval. The wind sens

In [95]:
import numpy as np
import pandas as pd
import scipy as sp
import math
rng=np.random.default_rng()

Функция bhep_test_statistic вычисляет BHEP-статистику:

In [96]:
def bhep_test_statistic(data, smoothing_parameter=1):
    n=data.shape[0]
    d=data.shape[1]
    if n <= d:
        print('not enough data!')
    S_n = np.cov(data, rowvar=False, bias=True)
    S_n_inverse_square_root = sp.linalg.sqrtm(np.linalg.inv(S_n))
    y_n=np.ma.dot((data-np.mean(data, axis=0)), S_n_inverse_square_root)
    jk_sum=0#считаем сумму по j,k из формулы тестовой статистики BHEP
    for j in range(n):
        for k in range(n):
            jk_sum+=np.exp(-(smoothing_parameter**2)*np.sum(np.square(data[j,:]-data[k,:]))/2)
    j_sum=0#считаем сумму по j из формулы тестовой статистики BHEP
    for j in range(n):
        j_sum+=np.exp(-(smoothing_parameter**2)*np.sum(np.square(data[j,:]))/(2*(1+smoothing_parameter**2)))
    bhep_test_statistic=(jk_sum/n) - (2*j_sum/((1+smoothing_parameter**2)**(d/2))) + n/((1+2*(smoothing_parameter**2))**(d/2))
    return bhep_test_statistic

Bhep_test использует симуляцию Монте-Карло -- генерирует по матрице ковариаций выборки многомерное нормальное распределение, берет sim_num выборок этого распределения, считает статистику для каждой выборки и сравнивает её с нашей выборкой. Функция выводит долю симуляций, имеющих меньшую статистику:

In [101]:
def bhep_test(data, smoothing_parameter=1, sim_num=300):
    n=data.shape[0]
    d=data.shape[1]
    d_cov = np.cov(data, rowvar=False, bias=True)#
    d_mean = data.mean(axis=0)
    if n <= d:
        print('not enough data!')
    a=[]
    for i in range(sim_num):
        print(f'progress={i}')
        a.append(bhep_test_statistic(rng.multivariate_normal(mean=d_mean, cov=d_cov, size=n), smoothing_parameter=smoothing_parameter))
    bh_stat=bhep_test_statistic(data=data, smoothing_parameter=smoothing_parameter)
    distr_func=0
    for i in a:
        if bh_stat>i:
            distr_func+=1
        if bh_stat==i:
            print('for some reason two statistics are the same')
    distr_func/=sim_num
    return distr_func, bh_stat

Проверяем детерминант -- если он очень близок к нулю, значит есть линейно зависимые переменные. В данном случае 0,009 -- весьма большой показатель -- до исключения других переменных он выдавал числа порядка e-32:

In [102]:
np.linalg.det(np.cov(data, rowvar=False, bias=True))

0.009397899066850294

Выборочная матрица ковариаций:

In [103]:
with np.printoptions(threshold=10000000):
    print(np.cov(data, rowvar=False, bias=True))

[[ 1.85778083e+01  1.59496876e+01  1.10982273e+01  2.05142579e+01
  -7.65162397e+00 -8.46405620e+00  4.00082934e+00  5.92835950e+00
   2.29725255e+01  3.86759244e+00  5.23381777e+00  7.13693926e+00
  -8.29632231e+00 -6.19463058e+00  3.34164137e+00 -9.98022603e+00
  -6.18767769e+00 -6.45285124e+00 -2.14174504e+01]
 [ 1.59496876e+01  2.25379300e+01 -1.09993636e+01  1.42739860e+01
  -6.49442769e+00 -4.71797479e+00  2.44025372e+00  3.99598347e+00
   1.49357481e+01  2.44705121e+00  3.26206653e+00  3.71449339e+00
   6.29652893e+00 -2.59976322e+00  8.64219643e-01 -5.35016777e+00
  -7.26664256e+00 -7.49640496e+00 -1.25054835e+01]
 [ 1.10982273e+01 -1.09993636e+01  8.62972727e+01  2.66138182e+01
  -7.12727273e+00 -1.66310455e+01  6.08654545e+00  5.07272727e+00
   2.92050000e+01  5.00304273e+00  7.12577273e+00  1.28035455e+01
  -4.63727273e+01 -1.54830000e+01  8.26877093e+00 -2.04090455e+01
   1.07954545e+00 -1.27272727e-02 -3.21522727e+01]
 [ 2.05142579e+01  1.42739860e+01  2.66138182e+01  2.58

Проверяем данные на многомерную нормальность:

In [104]:
bhep_test(data, sim_num=1000)

progress=0
progress=1
progress=2
progress=3
progress=4
progress=5
progress=6
progress=7
progress=8
progress=9
progress=10
progress=11
progress=12
progress=13
progress=14
progress=15
progress=16
progress=17
progress=18
progress=19
progress=20
progress=21
progress=22
progress=23
progress=24
progress=25
progress=26
progress=27
progress=28
progress=29
progress=30
progress=31
progress=32
progress=33
progress=34
progress=35
progress=36
progress=37
progress=38
progress=39
progress=40
progress=41
progress=42
progress=43
progress=44
progress=45
progress=46
progress=47
progress=48
progress=49
progress=50
progress=51
progress=52
progress=53
progress=54
progress=55
progress=56
progress=57
progress=58
progress=59
progress=60
progress=61
progress=62
progress=63
progress=64
progress=65
progress=66
progress=67
progress=68
progress=69
progress=70
progress=71
progress=72
progress=73
progress=74
progress=75
progress=76
progress=77
progress=78
progress=79
progress=80
progress=81
progress=82
progress=83
pr

progress=639
progress=640
progress=641
progress=642
progress=643
progress=644
progress=645
progress=646
progress=647
progress=648
progress=649
progress=650
progress=651
progress=652
progress=653
progress=654
progress=655
progress=656
progress=657
progress=658
progress=659
progress=660
progress=661
progress=662
progress=663
progress=664
progress=665
progress=666
progress=667
progress=668
progress=669
progress=670
progress=671
progress=672
progress=673
progress=674
progress=675
progress=676
progress=677
progress=678
progress=679
progress=680
progress=681
progress=682
progress=683
progress=684
progress=685
progress=686
progress=687
progress=688
progress=689
progress=690
progress=691
progress=692
progress=693
progress=694
progress=695
progress=696
progress=697
progress=698
progress=699
progress=700
progress=701
progress=702
progress=703
progress=704
progress=705
progress=706
progress=707
progress=708
progress=709
progress=710
progress=711
progress=712
progress=713
progress=714
progress=715

(0.565, 1.0006453135209674)

In [105]:
import numpy as np
import pandas as pd
import scipy as sp
import math
import itertools, random

pos_def_check проверяет положительную опреденность матрицы. Это нам понадобится, чтобы при градиентном спуске(методе Ньютона) не выйти за пределы мн-ва полож.-опред. матриц:

In [106]:
def pos_def_check(matrix):
    try:
        np.linalg.cholesky(matrix)
        return True
    except np.linalg.LinAlgError:
        return False

Далее я ввожу несколько переменных для проверки возможных ошибок алгоритма (после поправки кода они перестали появлятся, но лишними они не будут):

In [107]:
newton_descend_doesnt_work=0
newton_broke = 0#Эти два параметра используются для обнаружения случая, когда любой сдвиг по нашему методу выходит за пределы положительно-определенных матриц. Такого больше не происходит, но на всякий случай я эту проверку оставил

Запускаем алгоритм выбора ковариации:

In [108]:
data_sample=data
d=data_sample.shape[1]
n=data_sample.shape[0]
sample_cov_matrix=np.cov(data_sample,rowvar=False)
sample_prec_matrix=np.linalg.inv(sample_cov_matrix)
sample_corr_matrix=np.corrcoef(data_sample, rowvar=False)
graph=[]
for i in range (d):
    graph.append((i,i)) #граф, в который будем вершины добавлять -- добавляем(i,i), потому что далее нужно будет множество ребер графа и пар(i,i)
component_counter=1 #это чтобы знать, какой номер будет у новой компоненты связности, если мы создаем новую
found_pair=1 #счетчик, который проверяет, что мы нашли хотя бы одну вершину
#подбираем
epsilon=0.0000000001
approx_cov1_matrix=np.linalg.inv(np.diag(np.diag(np.linalg.inv(sample_cov_matrix)))) #это будет предыдущая по добавлению ребер ков. матрица, чтобы мы могли сравнить улучшение при добавлении каждого ребра
while found_pair!=0:
    found_pair=0
    delta_max=False #счетчик, который гарантирует, что мы выбираем ребро с наибольшим модулем корреляции
    g=itertools.combinations(range(d), 2)
    alist = list(g) 
    for a in alist:
        if a not in graph:#проверяем все возможные ребр --добавляем их по очереди
            approx_inv1_matrix=np.linalg.inv(approx_cov1_matrix)#начинаем с 
            graph.append(a)
            theta_a=[]#это цель итерационной процедуры -- найти матрицу, совпадающую (с учетом коэффициентов) с theta_a на элементах graph, и обратная которой равна 0 вне графа
            for i in range (len(graph)):
                if (graph[i][0]==graph[i][1]):
                    theta_a.append(-1/2*sample_cov_matrix[graph[i][0],graph[i][1]])
                else:
                    theta_a.append(-sample_cov_matrix[graph[i][0],graph[i][1]])
            theta_a=np.array(theta_a)
            #мы начнем с матрицы ковариаций, полученной на предыдущем шаге алгоритма -- мы добавляем ребро(т. е. убираем требования об условной нез-ми пары переменных), так что если предыдущая матрица удовлетворяла меньшему графу, она удовлетворяет и новому графу
            phi_i_a=[]#часть phi_i, которая соответсвует элементам graph
            for i in range (len(graph)):
                phi_i_a.append(approx_inv1_matrix[graph[i][0],graph[i][1]])
            phi_i_a=np.array(phi_i_a)
            approx_prec_matrix=approx_inv1_matrix.copy()#предыдущая матрица
            approx_cov_matrix=np.linalg.inv(approx_prec_matrix)
            theta_a_i=[]
            for i in range (len(graph)):
                if (graph[i][0]==graph[i][1]):
                    theta_a_i.append(-1/2*approx_cov_matrix[graph[i][0],graph[i][1]])
                else:
                    theta_a_i.append(-approx_cov_matrix[graph[i][0],graph[i][1]])
            theta_a_i=np.array(theta_a_i)
            delta=100
            epsilon=0.0000001
            while (delta>epsilon):#по сути градиентный поиск точки --здесь шаг такой, что при линейной функции мы бы сразу попали в искомую точку. Аллен это называет методом Ньютона для решения неявных уравнений
                #создаем матрицу Гамма_(1,1) -- составляем набор пар графа, считаем матрицу с ними, чтобы потом решить уравнение gamma_1_1_i*x=theta_a-theta_a_i и получить искомый сдвиг для phi_i_a
                gamma_1_1_i=np.zeros((len(graph), len(graph)))
                for i in range (len(graph)):
                    for j in range (len(graph)):
                        coef_mult=1
                        if (graph[i][0]==graph[i][1]):#коэффиценты отвечающие -1/2 у theta -- два минуса всегда обращаются в плюс, так что минусы игнорируем во всех случаях
                            coef_mult/=2
                        if (graph[j][0]==graph[j][1]):
                            coef_mult/=2
                        gamma_1_1_i[i,j]=coef_mult*(approx_cov_matrix[graph[i][0],graph[j][1]]*approx_cov_matrix[graph[i][1],graph[j][0]]+approx_cov_matrix[graph[i][0],graph[j][0]]*approx_cov_matrix[graph[i][1],graph[j][1]]) #спариваем элементы из i-й и j-й пары 2 способами
                s=np.linalg.lstsq(gamma_1_1_i,theta_a-theta_a_i)[0] #предупреждение каждый раз вылезает, но оно вроде не важное
                alpha=0.01
                proper_step=False
                while (proper_step==False):#проверяем, что шаг не слишком большой: матрица осталась положительно определенной
                    phi_i1_a=phi_i_a+alpha*s
                    #пересчитываем все итерационные величины по phi_i1_a
                    for i in range (len(graph)):
                        approx_prec_matrix[graph[i][0],graph[i][1]]=phi_i1_a[i]
                        approx_prec_matrix[graph[i][1],graph[i][0]]=phi_i1_a[i]
                    approx_cov_matrix=np.linalg.inv(approx_prec_matrix)
                    if pos_def_check(approx_cov_matrix)==True:
                        proper_step=True
                    else:
                        alpha/=10
                        if (alpha<0.00000000001):
                            proper_step='Broken'
                if (proper_step=='Broken'):#Код на тот случай, если что-то пошло не так и любой сдвиг, пропорциональный s, делает матрицу не положительно-определённой
                    newton_descend_doesnt_work+=1
                    newton_broke=1
                    break
                delta=np.sum(np.square(phi_i1_a-phi_i_a))#меряем размер шага в качестве критерия остановки
                phi_i_a=phi_i1_a
                for i in range (len(graph)):
                    if (graph[i][0]==graph[i][1]):
                        theta_a_i[i]=-1/2*approx_cov_matrix[graph[i][0],graph[i][1]]
                    else:
                        theta_a_i[i]=-approx_cov_matrix[graph[i][0],graph[i][1]]
            if newton_broke == 1:
                newton_broke = 0
                print('newton broke')
                break
            delta_k=n*(math.log(np.linalg.det(approx_cov1_matrix)) - math.log(np.linalg.det(approx_cov_matrix))) + (n-1)*(np.sum(sample_cov_matrix*(np.linalg.inv(approx_cov1_matrix)-np.linalg.inv(approx_cov_matrix))))#проверка на значимость
            if (1-sp.stats.chi2.cdf(delta_k, 1) < 0.05/(d*(d+1)/2-(len(graph)-1))):#делим на количество возможных дополнений к графу, чтобы шанс ложного принятия любого из рёбер был примерно 0,05
                if (delta_max==False) or (delta_k > delta_max):#эта реализация алгоритма перебирает все ребра и выбирает наилучшее из них
                    print(f'found candidate {graph[-1]}')
                    found_pair=graph[-1]
                    delta_max=delta_k 
                    approx_max_matrix=approx_cov_matrix#запоминаем ребро и итоговую матрицу ковариаций для ребра, дающего лучшие результаты, чем все предыдущие 
            graph.pop()#поскольку перебираем все ребра, надо удалить последнее добавленное ребро и попробовать остальные
    if found_pair!=0:
        graph.append(found_pair)
        print(graph)
        approx_cov1_matrix=approx_max_matrix#перебрав все ребра, мы выбираем оптимальное. Мы запомнили для него итоговую матрицу ковариций, так что её не надо заново искать
if len(graph)==d*(d+1)/2:
    full_graph+=1#индикатор того,что мы получили полный граф
print(f'Итоговый граф -- {graph}')

C:\Users\arkad\AppData\Local\Temp\ipykernel_9280\2228860477.py:58: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  s=np.linalg.lstsq(gamma_1_1_i,theta_a-theta_a_i)[0] #предупреждение каждый раз вылезает, но оно вроде не важное


found candidate (0, 1)
found candidate (0, 3)
found candidate (5, 13)
found candidate (5, 15)
found candidate (13, 15)
found candidate (16, 17)
[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (10, 10), (11, 11), (12, 12), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (18, 18), (16, 17)]
found candidate (0, 1)
found candidate (0, 3)
found candidate (5, 13)
found candidate (5, 15)
found candidate (13, 15)
[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (10, 10), (11, 11), (12, 12), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (18, 18), (16, 17), (13, 15)]
found candidate (0, 1)
found candidate (0, 3)
found candidate (5, 13)
found candidate (5, 15)
[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (10, 10), (11, 11), (12, 12), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (18, 18), (16, 17), (13, 15), (5, 15)]
found candidate (0, 1)
found candidate (0, 3)
found candidate (8, 10)
[

In [78]:
graph

[(0, 0),
 (1, 1),
 (2, 2),
 (3, 3),
 (4, 4),
 (5, 5),
 (6, 6),
 (7, 7),
 (8, 8),
 (9, 9),
 (10, 10),
 (11, 11),
 (12, 12),
 (13, 13),
 (14, 14),
 (15, 15),
 (16, 16),
 (17, 17),
 (18, 18),
 (16, 17),
 (13, 15),
 (5, 15),
 (8, 10),
 (10, 11),
 (0, 3),
 (15, 18),
 (10, 18),
 (5, 6),
 (4, 5),
 (3, 15),
 (4, 17),
 (0, 1),
 (10, 14),
 (14, 18),
 (7, 9),
 (7, 17),
 (2, 13),
 (1, 2),
 (2, 3),
 (1, 17)]

In [50]:
np.corrcoef(data, rowvar=False)#матрица коэффициентов корреляции

array([[ 1.00000000e+00,  7.79467354e-01,  2.77177415e-01,
         9.35409567e-01, -7.18703143e-01, -6.99224618e-01,
         7.06618172e-01,  6.42202142e-01,  7.13328408e-01,
         4.00103146e-01,  6.81115838e-01,  6.56913289e-01,
        -2.36646669e-01, -6.08430730e-01,  2.26648097e-01,
        -7.05072965e-01, -6.95955894e-01, -7.16946048e-01,
        -7.66995689e-01],
       [ 7.79467354e-01,  1.00000000e+00, -2.49408937e-01,
         5.90923808e-01, -5.53830713e-01, -3.53862071e-01,
         3.91300123e-01,  3.93007761e-01,  4.21063874e-01,
         2.29834143e-01,  3.85421042e-01,  3.10410113e-01,
         1.63063293e-01, -2.31830062e-01,  5.32177607e-02,
        -3.43163676e-01, -7.42041267e-01, -7.56185050e-01,
        -4.06598587e-01],
       [ 2.77177415e-01, -2.49408937e-01,  1.00000000e+00,
         5.63056383e-01, -3.10611839e-01, -6.37465119e-01,
         4.98774622e-01,  2.54963161e-01,  4.20762174e-01,
         2.40139818e-01,  4.30262059e-01,  5.46795583e-01,
    

In [1]:
rootgrp.close()

NameError: name 'rootgrp' is not defined